In [ ]:
# Disable logging

import logging

for logger_name in ["ragger", "sentence_transformers", "httpx"]:
    logging.getLogger(logger_name).setLevel("CRITICAL")

In [ ]:
# Load environment variables

from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
# Reset stores

from pathlib import Path

from ragger.document_store import JsonlDocumentStore, PostgresDocumentStore
from ragger.embedding_store import NumpyEmbeddingStore, PostgresEmbeddingStore

Path("feedback.db").unlink(missing_ok=True)

JsonlDocumentStore().remove()
NumpyEmbeddingStore().remove()
with PostgresDocumentStore()._connect() as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS documents")
with PostgresEmbeddingStore()._connect() as conn:
    cursor = conn.cursor()
    cursor.execute("DROP TABLE IF EXISTS embeddings")

# Ragger Demo

## Installation

Installation with `pip`:

In [ ]:
# !pip install "ragger[postgres]@git+ssh://git@github.com/alexandrainst/ragger.git"

Installation with `uv`:

In [ ]:
# !uv add git+ssh://git@github.com/alexandrainst/ragger.git --extra postgres

You can replace the `all` extra with any combination of the following, to install only
the components you need:

- `onprem_cpu`
- `onprem_gpu`
- `keyword_search`
- `postgres`
- `demo`

For `pip`, this is done by comma-separating the extras (e.g., `ragger[onprem_cpu,demo]`), 
while for `uv`, you add multiple `--extra` flags (e.g., `--extra onprem_cpu --extra demo`).

## Quick Start

Initialise a RAG system with default settings as follows:

In [ ]:
from ragger import RagSystem

rag_system = RagSystem()
rag_system

Normally we would have an existing document store that we hook up our RAG system to. You can also create your own.

For now, we'll just manually add some documents with the `add_documents` method, which also adds the embeddings to the embedding store:

In [ ]:
# Can also be a list of dictionaries or a list of `ragger.data_models.Document` objects
documents = [
    "København er hovedstaden i Danmark.",
    "Danmark har 5,8 millioner indbyggere.",
    "Danmark er medlem af Den Europæiske Union.",
]

In [ ]:
rag_system.add_documents(documents)
rag_system

To answer a query we use the `answer` method:

In [ ]:
answer, supporting_documents = rag_system.answer("Hvor mange bor der i Danmark?")
print(f"Answer: {answer!r}")
print(f"Sources: {supporting_documents}")

We can also use the convenience method `answer_formatted` to get a HTML-formatted answer with both the answer and sources:

In [ ]:
from IPython.display import HTML

while True:
    query = input("Question ('q' to exit): ").lower()
    if query == "q":
        break
    answer = rag_system.answer_formatted(query)
    display(HTML(answer))

A working demo can be run using the `Demo` class:

In [ ]:
from ragger import Demo

demo = Demo(rag_system=rag_system)
demo.launch()

This demo collects thumbs up/down feedback and stores it to a local SQLite database. Furthermore, the demo can be persisted on the Hugging Face Hub by setting the `persistent_sharing_config` in the `Demo` initialisation.

## Batteries Included

Ragger supports the following components:

### Document Stores

These are the databases carrying all the documents. Documents are represented as objects
of the `Document` data class, which has an `id` and a `text` field. These can all be
imported from `ragger.document_store`.

- `JsonlDocumentStore`: A document store that reads from a JSONL file. (default)
- `SqliteDocumentStore`: A document store that uses a SQLite database to store documents.
- `PostgresDocumentStore`: A document store that uses a PostgreSQL database to store
  documents. This assumes that the PostgreSQL server is already running.
- `TxtDocumentStore`: A document store that reads documents from a single text file,
  separated by newlines.

### Retrievers

Retrievers are used to retrieve documents from the document store that are relevant to a given query. These can all be imported from `ragger.retriever`.

- `EmbeddingRetriever`: A retriever that embeds documents and retrieves them through k-nearest neighbours.

  You can choose between the following embedders, via the `embedder` argument:

  - `OpenAIEmbedder`: An embedder that uses the OpenAI Embeddings API. (default)
  - `E5Embedder`: An embedder that uses an E5 model.

  You can choose between the following embedding stores, via the `embedding_store` argument:

  - `NumpyEmbeddingStore`: An embedding store that stores embeddings in a NumPy array.
  (default)
  - `PostgresEmbeddingStore`: An embedding store that uses a PostgreSQL database to store
    embeddings, using the `pgvector` extension. This assumes that the PostgreSQL server is
    already running, and that the `pgvector` extension is installed. See
    [here](https://github.com/pgvector/pgvector?tab=readme-ov-file#installation) for more
    information on how to install the extension.

- `BM25Retriever`: A retriever that uses the keyword-based BM25 algorithm.
- `HybridRetriever`: A retriever that combines several retrievers.

### Generators

Generators are used to generate answers from the retrieved documents and the question.
These can all be imported from `ragger.generator`.

- `OpenAIGenerator`: A generator that uses the OpenAI API. (default)
- `VllmGenerator`: A generator that uses vLLM to wrap almost any model from the Hugging
  Face Hub.

### Using non-default components

Here is an example where we're using a Postgres server for both the document store and embedding store:

In [ ]:
from ragger.document_store import PostgresDocumentStore
from ragger.retriever import EmbeddingRetriever
from ragger.embedding_store import PostgresEmbeddingStore

In [ ]:
postgres_rag_system = RagSystem(
    document_store=PostgresDocumentStore(), 
    retriever=EmbeddingRetriever(
        embedding_store=PostgresEmbeddingStore()
    ),
)
postgres_rag_system.add_documents(documents)

In [ ]:
answer, supporting_documents = postgres_rag_system.answer(
    "Hvad er hovedstaden i Danmark?"
)
print(f"Answer: {answer!r}")
print(f"Sources: {supporting_documents}")

## Hackable

You can also create custom components by subclassing the following classes:

- `DocumentStore`
- `Retriever` (and by extension, also `Embedder` and `EmbeddingStore`)
- `Generator`

These can then simply be added to a `RagSystem`. Here is a minimal example:

In [ ]:
import typing

from ragger import Document, DocumentStore, Index


class InMemoryDocumentStore(DocumentStore):
    """A document store that just keeps all documents in memory."""

    def __init__(self, documents: list[str]):
        self.documents = [
            Document(id=str(i), text=text) for i, text in enumerate(documents)
        ]

    def add_documents(self, documents: typing.Iterable[Document]):
        self.documents.extend(documents)

    def remove(self):
        self.documents = []

    def __getitem__(self, index: Index) -> str:
        return self.documents[int(index)]

    def __contains__(self, index: Index) -> bool:
        return index in {doc.id for doc in self.documents}

    def __iter__(self) -> typing.Generator[Document, None, None]:
        yield from self.documents

    def __len__(self) -> int:
        return len(self.documents)


document_store = InMemoryDocumentStore(documents=documents)
document_store

In [ ]:
in_memory_rag_system = RagSystem(document_store=document_store)
in_memory_rag_system

In [ ]:
answer, supporting_documents = in_memory_rag_system.answer("Hvad er hovedstaden i Danmark?")
print(f"Answer: {answer!r}")
print(f"Sources: {supporting_documents}")